In [1]:
import pandas as pd
import json

In [2]:
def load_merged_popqa(filepath):
    records = []
    with open(filepath, 'r') as f:
        for line in f:
            obj = json.loads(line)
            doc = obj.get("doc", {})
            records.append({
                "subject_qid": doc.get("s_uri"),
                "answer_qid": doc.get("o_uri"),
                "exact_match": obj.get("exact_match", 0.0),
                "s_pop": doc.get("s_pop"),
                "o_pop": doc.get("o_pop"),
                "subject_num_chunks": doc.get("subject_num_chunks"),
                "answer_num_chunks": doc.get("answer_num_chunks"),
                "num_shared_chunks": doc.get("num_shared_chunks"),
            })
    return pd.DataFrame(records)

In [ ]:
df_burst_100 = load_merged_popqa("/home/morg/students/gottesman3/knowledge-analysis-suite/olmes/hp_final/kas_1B_burst_100/task-000-popqa_cloze-merged-results.jsonl")
df_burst_1 = load_merged_popqa("/home/morg/students/gottesman3/knowledge-analysis-suite/olmes/hp_final/kas_1B_burst_1/task-000-popqa_cloze-merged-results.jsonl")
df_baseline = load_merged_popqa("/home/morg/students/gottesman3/knowledge-analysis-suite/olmes/hp_final/kas_1B_1/task-000-popqa_cloze-merged-results.jsonl")

import pickle

with open("/home/morg/students/gottesman3/knowledge-analysis-suite/OLMo-core/chunk_ordering_data/rare_disjoint_entities_50-100.pkl", "rb") as f:
    data = pickle.load(f)

entity_names = [x["entity_name"] for x in data]

from datasets import load_dataset
ds = load_dataset("dhgottesman/popqa-kas", split="train")
dataset = ds.to_pandas()

filtered = dataset[dataset["s_wiki_title"].isin(entity_names)]
filtered = filtered.rename(columns={"s_uri": "subject_qid"})

print(df_burst_1.merge(filtered, on="subject_qid")["exact_match"].mean())
print(df_burst_100.merge(filtered, on="subject_qid")["exact_match"].mean())
print(df_baseline.merge(filtered, on="subject_qid")["exact_match"].mean())

0.05776173285198556
0.05415162454873646
0.07220216606498195


In [9]:
entity_ids = [x["entity_id"] for x in data]
entity_ids

[74667,
 1015075,
 1398044,
 1780270,
 259529,
 1937090,
 82583,
 96949,
 2516414,
 2897727,
 811017,
 2868452,
 1329022,
 1327426,
 647117,
 2826388,
 2433530,
 1344682,
 953037,
 993495,
 2033555,
 2334061,
 1168664,
 2322771,
 83133,
 707834,
 73117,
 2917477,
 2059559,
 537812,
 2517532,
 2886218,
 33408,
 2921735,
 1120846,
 321179,
 1317890,
 176111,
 2834959,
 876454,
 571503,
 1180944,
 102363,
 2939623,
 1122700,
 438280,
 1601510,
 288631,
 329546,
 2832813,
 114759,
 1610110,
 336488,
 934373,
 1136189,
 939739,
 2322482,
 225156,
 2929832,
 2325270,
 1149571,
 1090589,
 312292,
 515039,
 955351,
 849390,
 299598,
 2586788,
 2083978,
 2245545,
 293883,
 1235945,
 535203,
 37422,
 2888874,
 952835,
 2906660,
 713338,
 775672,
 736008,
 151730,
 707906,
 2316382,
 2511556,
 428092,
 245641,
 321862,
 2078768,
 1678786,
 405175,
 2147967,
 2882715,
 2007264,
 1239963,
 705621,
 973494,
 655198,
 461217,
 2902535,
 854105,
 549514,
 2046870,
 798287,
 1851794,
 1423896,
 1308066

In [10]:
filtered = dataset[dataset["subj_id"].isin(entity_ids)]
filtered = filtered.rename(columns={"s_uri": "subject_qid"})

print(df_burst_1.merge(filtered, on="subject_qid")["exact_match"].mean())
print(df_burst_100.merge(filtered, on="subject_qid")["exact_match"].mean())
print(df_baseline.merge(filtered, on="subject_qid")["exact_match"].mean())

0.036418816388467376
0.024279210925644917
0.042488619119878605


In [11]:
filtered

,id,subj,prop,obj,subj_id,prop_id,obj_id,s_aliases,o_aliases,subject_qid,...,s_pop,o_pop,question,possible_answers,subject_chunks,subject_num_chunks,answer_chunks,answer_num_chunks,shared_chunks,num_shared_chunks
20,4419938,Rudolf Hilferding,occupation,politician,1942931,22,2834605,[Rudolph Hilferding],"[political leader, political figure, polit., pol]",Q57848,...,2092,25692,What is Rudolf Hilferding's occupation?,"[economist, economists, politician, political ...","[1430330, 2733487, 3487218, 3974505, 3974506, ...",84,"[2955255, 2989263, 3086390, 1922585, 6357886, ...",12899,[],0
23,2605479,Richard Martin,occupation,actor,1122939,22,1114174,[],"[actress, actors, actresses]",Q3431024,...,381,81374,What is Richard Martin's occupation?,"[actor, actress, actors, actresses]","[174316, 3365371, 4359817, 4772661, 4777247, 4...",67,"[3603921, 739103, 2483278, 2809303, 7461360, 1...",12938,[],0
30,857817,Heath,occupation,songwriter,364481,22,2577376,[Hiroshi Morie],[song writer],Q1602025,...,1132,25550,What is Heath's occupation?,"[songwriter, song writer]","[2024031, 2589666, 3808608, 3808609, 8123830, ...",63,"[3674679, 3931115, 7600991, 7947947, 2369818, ...",2923,[],0
44,2224150,Brendan Canty,occupation,musician,971262,22,2133594,[Brendan John Canty],[musical artist],Q2924425,...,2680,20148,What is Brendan Canty's occupation?,"[composer, musician, musical artist]","[3182572, 4110015, 3872910, 218745, 999908, 15...",77,"[3825464, 3823249, 10427871, 1986834, 8723189,...",2626,[],0
73,1094065,Jilly Cooper,occupation,journalist,474011,22,663400,[],"[journo, journalists]",Q1689012,...,3857,24952,What is Jilly Cooper's occupation?,"[journalist, journo, journalists, novelist]","[2492713, 2492714, 4007851, 6478281, 8952841, ...",74,"[1537456, 6949534, 9645967, 5606567, 826410, 4...",7672,[4018062],1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14155,5407282,Polkowice County,capital,Polkowice,2403912,422,836383,[],[],Q715538,...,386,776,What is the capital of Polkowice County?,[Polkowice],"[1224476, 2577749, 3501888, 3502782, 3504707, ...",82,"[4282281, 6902893, 8668681, 1214507, 1224476, ...",60,"[8670852, 8668681, 8664084, 8666902, 1224476, ...",34
14165,1505834,Sangrur district,capital,Sangrur,671766,422,828040,[],[],Q1945515,...,2685,2869,What is the capital of Sangrur district?,[Sangrur],"[5122575, 16127, 2520408, 8643815, 9683574, 10...",92,"[6418056, 588947, 2077382, 7177197, 7431855, 7...",168,"[6233345, 2536069, 93702, 5581573, 4684298, 45...",53
14172,3447132,Worth County,capital,Sylvester,1487396,422,934740,"[Worth County, Georgia]","[Sylvester, Georgia]",Q486154,...,897,933,What is the capital of Worth County?,"[Sylvester, Sylvester, Georgia]","[1408138, 5331541, 9707896, 1335920, 3094468, ...",54,"[1912381, 2424221, 7730093, 7067417, 9731329, ...",44,"[3094370, 5209126, 5209127, 1408138, 9701003, ...",14
14177,6403523,Northern Province,capital,Makeni,2883504,422,1292636,"[The Northern Province, Northern Sierra Leone,...",[],Q912359,...,751,1025,What is the capital of Northern Province?,[Makeni],"[1406552, 10353484, 2428885, 2428886, 1981429,...",93,"[346524, 2715622, 3552368, 3574838, 3589224, 4...",106,"[6009481, 7334796, 8062991, 8994965, 5557275, ...",28


In [4]:
df_burst_100

,subject_qid,answer_qid,exact_match,s_pop,o_pop,subject_num_chunks,answer_num_chunks,num_shared_chunks
0,Q5543720,Q82955,0.0,142,25692,8,12899,0
1,Q6247345,Q1930187,0.0,236,24952,19,7672,0
2,Q5725578,Q82955,0.0,58,25692,7,12899,0
3,Q6377295,Q82955,0.0,127,25692,3,12899,1
4,Q5354261,Q1114448,0.0,317,9649,14,2769,2
...,...,...,...,...,...,...,...,...
14262,Q55,Q727,1.0,416350,147710,124221,26494,11413
14263,Q3441823,Q1012,NaN,335,4086,45,549,43
14264,Q1353,Q987,0.0,219227,128193,19175,13134,2727
14265,Q407199,Q1218,0.0,17545,136734,2046,24068,520


In [5]:
data

[{'entity_id': 74667,
  'entity_name': 'Control',
  'num_chunks': 65,
  'chunks': [2237061,
   2611446,
   2954313,
   3119621,
   3260500,
   3286712,
   3398897,
   3398898,
   4777376,
   7641034,
   10348211,
   3577493,
   2824558,
   4573211,
   4562086,
   5665654,
   7127532,
   8283741,
   9279434,
   4076895,
   3751845,
   1596766,
   8814376,
   4050967,
   1555654,
   1613032,
   2436402,
   4462698,
   10255910,
   954898,
   2439952,
   7281610,
   7615497,
   3918203,
   1961042,
   4010198,
   5109440,
   7869780,
   7926912,
   8030370,
   8294777,
   8463929,
   9896865,
   4050968,
   4281465,
   7103624,
   2319448,
   4202237,
   8967952,
   7484676,
   2269237,
   10002282,
   8953533,
   4010199,
   3431988,
   8368567,
   1926549,
   2237065,
   2434318,
   6945337,
   8225483,
   2319449,
   8368566,
   3623433,
   7484675],
  'chunks_lengths': [2048,
   2048,
   2048,
   2048,
   2048,
   2048,
   2048,
   2048,
   2048,
   2048,
   2048,
   2044,
   2042,
  

In [6]:
dataset

,id,subj,prop,obj,subj_id,prop_id,obj_id,s_aliases,o_aliases,s_uri,...,s_pop,o_pop,question,possible_answers,subject_chunks,subject_num_chunks,answer_chunks,answer_num_chunks,shared_chunks,num_shared_chunks
0,4222362,George Rankin,occupation,politician,1850297,22,2834605,[George James Rankin],"[political leader, political figure, polit., pol]",Q5543720,...,142,25692,What is George Rankin's occupation?,"[politician, political leader, political figur...","[7685081, 9599765, 8604880, 6064689, 4571913, ...",8,"[2955255, 2989263, 3086390, 1922585, 6357886, ...",12899,[],0
1,4725190,John Mayne,occupation,journalist,2079053,22,663400,[],"[journo, journalists]",Q6247345,...,236,24952,What is John Mayne's occupation?,"[journalist, journo, journalists]","[2876222, 2924720, 3371074, 3789637, 4017314, ...",19,"[1537456, 6949534, 9645967, 5606567, 826410, 4...",7672,[],0
2,4382392,Henry Feilden,occupation,politician,1925450,22,2834605,[Henry Master Feilden],"[political leader, political figure, polit., pol]",Q5725578,...,58,25692,What is Henry Feilden's occupation?,"[politician, political leader, political figur...","[8241472, 8241475, 2951364, 6282153, 8237327, ...",7,"[2955255, 2989263, 3086390, 1922585, 6357886, ...",12899,[],0
3,4822110,Kathy Saltzman,occupation,politician,2122743,22,2834605,[],"[political leader, political figure, polit., pol]",Q6377295,...,127,25692,What is Kathy Saltzman's occupation?,"[politician, political leader, political figur...","[9159492, 9046628, 9046629]",3,"[2955255, 2989263, 3086390, 1922585, 6357886, ...",12899,[9159492],1
4,4011112,Eleanor Davis,occupation,cartoonist,1752619,22,68412,[Eleanor McCutcheon Davis],"[graphic artist, animator, illustrator]",Q5354261,...,317,9649,What is Eleanor Davis's occupation?,"[cartoonist, graphic artist, animator, illustr...","[7538116, 7538117, 8457, 281353, 1051849, 2626...",14,"[1700853, 1674837, 3989776, 4214444, 5666220, ...",2769,"[4658248, 2626714]",2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14262,4165948,Netherlands,capital,Amsterdam,1825504,422,2455815,"[Holland, the Netherlands, NL, NED, Nederland,...","[Mokum, Amsterdam, NL, Amsterdam, Netherlands,...",Q55,...,416350,147710,What is the capital of Netherlands?,"[The Hague, Den Haag, 's-Gravenhage, Haag, Ams...","[5777962, 6011663, 10005, 12467, 14972, 15972,...",124221,"[2418043, 2761536, 8250372, 8636787, 2227, 792...",26494,"[3112964, 2195465, 3178507, 7274507, 4358159, ...",11413
14263,2617063,City Municipality of Celje,capital,Celje,1127153,422,5035,[Celje city municipality],[],Q3441823,...,335,4086,What is the capital of City Municipality of Ce...,[Celje],"[11876, 3473216, 6594944, 6976486, 6976780, 69...",45,"[8239740, 4555967, 27062, 139071, 196268, 9289...",549,"[6594944, 6978948, 6981513, 6983817, 6976780, ...",43
14264,471864,Delhi,capital,New Delhi,192624,422,2938512,"[National Capital Region of Delhi, NCR]","[New Delhi district, Nayi Dilli]",Q1353,...,219227,128193,What is the capital of Delhi?,"[New Delhi, New Delhi district, Nayi Dilli]","[3025967, 3444851, 3702722, 3702723, 3809404, ...",19175,"[10380946, 10380947, 91369, 93632, 203436, 210...",13134,"[1507331, 3596291, 6111238, 4489227, 7176203, ...",2727
14265,2965573,Palestinian territories,capital,Jerusalem,1270077,422,127625,"[occupied Palestinian territories, OPT, West B...","[Yerushalayim, J'lem, Aelia Capitolina, Al-Qud...",Q407199,...,17545,136734,What is the capital of Palestinian territories?,"[Jerusalem, Yerushalayim, J'lem, Aelia Capitol...","[6442497, 10065169, 714310, 344403, 747518, 10...",2046,"[6464599, 2607716, 2322376, 3386360, 3401006, ...",24068,"[3512322, 4020230, 1835021, 3174413, 4380689, ...",520
